# Init

In [96]:
#import
import numpy as np 
import random as npr

# Variable
eta = 0.5;

global xList;
xList = []
global dList;
dList = []

global eList
eList = []
global biasList
biasList = [];
global wMatrix
wMatrix = [];
global vMatrix
vMatrix = []
global yMatrix
yMatrix = [];
global sigmaMatrix
sigmaMatrix = [];
global nodeIndexMatrix
nodeIndexMatrix = [];

## Functions

### Auxilary Function

In [97]:
def getNodeLayer(nodeNum:int):
    result = 0;
    for layer in nodeIndexMatrix:
        if nodeNum in layer: return result
        result += 1
    raise RuntimeError("no node " + nodeNum + "existed in nodeIdexMatrix");

def getNodeLayerNIndex(nodeNum:int):
    nodeLayer = getNodeLayer(nodeNum)
    nodeIndex = nodeIndexMatrix[nodeLayer].index(nodeNum)
    return nodeLayer, nodeIndex

### Calculation Function

In [98]:
def vFunc(nodeNum:int):
    nodeLayer, nodeIndex = getNodeLayerNIndex(nodeNum);
    if nodeLayer == 0:
        return 0;
    else:
        pastIndexes = nodeIndexMatrix[nodeLayer-1];
        pastWs = list(map(lambda x: wMatrix[x][nodeIndex], pastIndexes))
        pastYs = yMatrix[nodeLayer-1]
        vlist = list(map(lambda x,y: x*y, pastYs,pastWs))
        bias = biasList[nodeNum];
        v = sum(vlist)+bias;
        return v;


def simgFunc(input:int):
    return 1/(1 + np.exp(-input));

def simgPrimeFunc(input:int):
    return simgFunc(input)*(1-simgFunc(input));

def yFunc(nodeNum:int):
    nodeLayer, nodeIndex = getNodeLayerNIndex(nodeNum);
    # print("y index of:" + str(nodeNum))
    if nodeLayer == 0:
        return xList[nodeNum];
    else:
        v = vMatrix[nodeLayer][nodeIndex];
        return simgFunc(v);

def eFunc(nodeNum:int):
    nodeLayer, nodeIndex = getNodeLayerNIndex(nodeNum);
    if nodeLayer == len(nodeIndexMatrix)-1:
        y = yMatrix[nodeLayer][nodeIndex]
        d = dList[nodeIndex]
        return d - y;
    else:
        raise RuntimeError("Can't calculate e("+str(nodeNum)+")")

def sigmaFunc(nodeNum:int):
    nodeLayer, nodeIndex = getNodeLayerNIndex(nodeNum);
    phiPrime = simgPrimeFunc(vFunc(nodeNum))
    if nodeLayer == len(nodeIndexMatrix)-1:
        e = eFunc(nodeNum);
        return e*phiPrime;
    else:
        forWs = wMatrix[nodeNum];
        forSigmas = sigmaMatrix[nodeLayer+1]
        sumSigW = sum(map(lambda x,y: x*y,forWs, forSigmas));
        return phiPrime*sumSigW;

def deltaWFunc(nodeI:int, nodeJ:int):
    nodeILayer, nodeIIndex = getNodeLayerNIndex(nodeI);
    nodeJLayer, nodeJIndex = getNodeLayerNIndex(nodeJ);
    yI = yMatrix[nodeILayer][nodeIIndex];
    if nodeJLayer == len(nodeIndexMatrix)-1:
        e = eFunc(nodeJ);
        phiPrime = simgPrimeFunc(vFunc(nodeJ))
        return eta*e*phiPrime*yI;
    else:
        sigmaJ = sigmaMatrix[nodeJLayer][nodeJIndex];
        return eta*sigmaJ*yI;

def getNewWs(nodeNum:int):
    nodeLayer, nodeIndex = getNodeLayerNIndex(nodeNum);
    if nodeLayer == len(nodeIndexMatrix)-1:
        raise RuntimeError("Cann't calculate delta-Ws of output node:" + str(nodeNum))
    forNodeIndexes = nodeIndexMatrix[nodeLayer+1];
    oldWs = wMatrix[nodeNum]
    deltaWs = list(map(lambda x: deltaWFunc(nodeNum, x), forNodeIndexes))
    newWs = list(map(lambda x,y: x+y, oldWs,deltaWs))
    return newWs;

def getNewBias(nodeNum:int):
    nodeLayer, nodeIndex = getNodeLayerNIndex(nodeNum);
    if nodeLayer == 0: raise RuntimeError("No bias for node:"+str(nodeNum))
    sigma = sigmaMatrix[nodeLayer][nodeIndex];
    oldBias = biasList[nodeNum];
    newBias = oldBias+(eta*sigma);
    return newBias;



# Training Ground

## Training Function

In [99]:
def initAllContainer(layerNodeCount:list):
    #Clear
    xList.clear();
    dList.clear();
    eList.clear();
    biasList.clear();
    wMatrix.clear();
    vMatrix.clear()
    yMatrix.clear();
    sigmaMatrix.clear();
    nodeIndexMatrix.clear();
    #Create 
    currIndex = 0;
    for l in layerNodeCount:
        nodeIndexMatrix.append(list(range(currIndex,currIndex+l)))
        yMatrix.append([0]*l)
        vMatrix.append([0]*l)
        sigmaMatrix.append([0]*l)
        currIndex += l
    for layer in range(len(nodeIndexMatrix)):
        for node in nodeIndexMatrix[layer]:
            if not node in nodeIndexMatrix[len(nodeIndexMatrix)-1]:
                wMatrix.append([0]*len(nodeIndexMatrix[layer+1]))
    for _ in range(max(max(nodeIndexMatrix))+1):
        eList.append(0);
        biasList.append(0);

def UpdateRandomWMatrix(minVal:float, maxVal:float):
    for i in range(len(wMatrix)):
        for j in range(len(wMatrix[i])):
            wMatrix[i][j] = round(npr.uniform(minVal, maxVal),2)            

def UndateRandomBias(minVal:float, maxVal:float):
    for i in range(len(biasList)):
        biasList[i] = round(npr.uniform(minVal, maxVal),2)   

# def getXnDList(textsList:list, dataIndex:int):
#     classOrder = ["cp","im","pp","imU","om","omL","imL","imS"];
#     dataText = textsList[dataIndex];
#     dataList = dataText.split();
#     clas = dataList.pop(7)
#     xL = dataList
#     dL = [0]*8;
#     dL[classOrder.index(clas)] = 1;
#     return xL, dL;

def updateAllY():
    for layNum in range(len(nodeIndexMatrix)):
        nodelist = nodeIndexMatrix[layNum]
        vList = list(map(vFunc, nodelist));
        vMatrix[layNum] = vList
        yList = list(map(yFunc, nodelist));
        yMatrix[layNum] = yList

def backPropergate():
    reverseLayNum = list(range(len(nodeIndexMatrix))).copy()
    reverseLayNum.pop(0)
    reverseLayNum.reverse()
    for layNum in reverseLayNum:
        nodelist = nodeIndexMatrix[layNum]
        sigmaList = list(map(sigmaFunc, nodelist));
        sigmaMatrix[layNum] = sigmaList
    indexesForW = nodeIndexMatrix.copy()
    indexesForW.pop(len(nodeIndexMatrix)-1)
    for node in range(max(max(indexesForW))):
        newWs = getNewWs(node);
        wMatrix[node] = newWs;
    startBiasIndex = max(nodeIndexMatrix[0])
    for b in range(startBiasIndex+1,max(max(nodeIndexMatrix))+1):
        newBias = getNewBias(b);
        biasList[b] = newBias;


## Execution

In [100]:
def getResultFromY(yList:list, roundabling:bool):
    if(roundabling):
        return list(map(round,yList));
    else:
        return yList;
    
def getTrainNTestIndexes(rowCount:int):
    allDataIndexes = range(0,rowCount)
    trainDataIndexes = list(npr.sample(range(rowCount), round(rowCount*0.9)))
    testDataIndexes = [x for x in allDataIndexes if x not in trainDataIndexes ]
    return trainDataIndexes, testDataIndexes

def getNNErr(dataPath:str, NNDimen:list, fold:int):
    fil = open(dataPath, "r")
    texts = fil.readlines()
    rowCount = len(texts);

    initAllContainer(NNDimen);
    UpdateRandomWMatrix(0.01,0.99);
    UndateRandomBias(0.01,0.99);
    
    # print(nodeIndexMatrix);
    # print();
    # print(wMatrix);
    # print();
    # print(biasList);
    errList = []
    for _ in range(fold):
        # Train
        trainIndexes, testIndexes = getTrainNTestIndexes(rowCount);
        # print("train with " + str(len(trainIndexes))+" indexes")
        # print("test with " + str(len(testIndexes))+" indexes")
        for i in trainIndexes:

            classOrder = ["cp","im","pp","imU","om","omL","imL","imS"];
            dataText = texts[i];
            dataList = dataText.split();
            clas = dataList.pop(7)
            global xList
            xList = list(map(float, dataList))
            dL = [0]*8;
            dL[classOrder.index(clas)] = 1;
            global dList
            dList = dL

            updateAllY();
            backPropergate();
        # Evalua
        errCount = 0;
        for i in testIndexes:
            classOrder = ["cp","im","pp","imU","om","omL","imL","imS"];
            dataText = texts[i];
            dataList = dataText.split();
            clas = dataList.pop(7)
            dL = [0]*8;
            dL[classOrder.index(clas)] = 1;
            dTestList = dL

            updateAllY();
            rawY = yMatrix[len(nodeIndexMatrix)-1]
            result = getResultFromY(rawY, True);
            if (not (dTestList==result)): errCount += 1;
        errList.append(errCount);

    return errList;

# Evaluation

In [101]:
result = getNNErr("ProcessedData/type1data.data", [7,8,8],10);
print(result);

[34, 22, 34, 21, 23, 34, 34, 28, 19, 30]
